In [1]:
from flask import Flask, request, jsonify, make_response
import cv2
from ultralytics import YOLO
from paddleocr import PaddleOCR, draw_ocr
import matplotlib.pyplot as plt
import json
import subprocess
# from flask_cors import CORS
import os

app = Flask(__name__)

# CORS(app, resources={r"/predict": {"origins": "*"}})

class_mapping = {
    0: 'BuyerAddress',
    1: 'BuyerName',
    2: 'BuyerTaxID',
    3: 'Date',
    4: 'InvoiceNumber',
    5: 'ProductGrossWorth',
    6: 'ProductName',
    7: 'ProductNetPrice',
    8: 'ProductNetWorth',
    9: 'ProductQuantity',
    10: 'ProductUM',
    11: 'ProductVAT',
    12: 'Seller IBAN',
    13: 'SellerAddress',
    14: 'SellerName',
    15: 'SellerTaxID',
    16: 'TotalGrossWorth',
    17: 'TotalNetWorth',
    18: 'TotalVAT',
    19: 'TotalVATPercent'
}

seg_model = YOLO('./seg_best.pt')
det_model = YOLO('./det_best.pt')

ocr = PaddleOCR(use_angle_cls=True, lang='en')

@app.after_request
def after_request(response):
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
    response.headers.add('Access-Control-Allow-Methods', 'GET,POST,PUT,DELETE,OPTIONS')
    return response

@app.route('/predict', methods=['POST'])
def inference():
    imag = request.files['image']
    if not imag:
        return jsonify({"error": "No imag provided"}), 400
    img = cv2.imread(imag)
    data_extracted = {
        'BuyerAddress': '',
        'BuyerName': '',
        'BuyerTaxID': '',
        'Date': '',
        'InvoiceNumber': '',
        'ProductGrossWorth': [],
        'ProductName': [],
        'ProductNetPrice': [],
        'ProductNetWorth': [],
        'ProductQuantity': [],
        'ProductUM': [],
        'ProductVAT': [],
        'Seller IBAN': '',
        'SellerAddress': '',
        'SellerName': '',
        'SellerTaxID': '',
        'TotalGrossWorth': '',
        'TotalNetWorth': '',
        'TotalVAT': '',
        'TotalVATPercent': ''
    }

    json_data = {
        'BuyerAddress': data_extracted['BuyerAddress'],
        'BuyerName': data_extracted['BuyerName'],
        'BuyerTaxID': data_extracted['BuyerTaxID'],
        'Date': data_extracted['Date'],
        'InvoiceNumber': data_extracted['InvoiceNumber'],
        'items': '',
        'Seller IBAN': data_extracted['Seller IBAN'],
        'SellerAddress': data_extracted['SellerAddress'],
        'SellerName': data_extracted['SellerName'],
        'SellerTaxID': data_extracted['SellerTaxID'],
        'TotalGrossWorth': data_extracted['TotalGrossWorth'],
        'TotalNetWorth': data_extracted['TotalNetWorth'],
        'TotalVAT': data_extracted['TotalVAT'],
        'TotalVATPercent': data_extracted['TotalVATPercent']
    }
    results_seg = seg_model(img)
    for i, box in enumerate(results_seg[0].boxes.xyxy):
        x1, y1, x2, y2 = map(int, box)
        roi = img[y1:y2, x1:x2]

        results_det = det_model(roi)
        detections = []

        for result in results_det:
            boxes = result.boxes
            for box in boxes:
                class_id = int(box.cls)
                class_name = class_mapping.get(class_id)

                if class_name:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cropped_img = img[y1:y2, x1:x2]
                    detections.append({
                        'class_name': class_name,
                        'bbox': [x1, y1, x2, y2],
                        'cropped_img': cropped_img
                    })

        detections = sorted(detections, key=lambda det: det['bbox'][1])

        for det in detections:
            class_name = det['class_name']
            cropped_img = det['cropped_img']

            print(f"\nOCR Results for {class_name}:")
            result = ocr.ocr(cropped_img, cls=True)
            res = result[0]
            text = ""
            if res is not None:
                for item in res:
                    text = text + " " + item[1][0]
                if class_name.startswith("Product"):
                    data_extracted[class_name].append(text.strip())
                else:
                    data_extracted[class_name] = text.strip()

    json_data.update({
        'BuyerAddress': data_extracted['BuyerAddress'],
        'BuyerName': data_extracted['BuyerName'],
        'BuyerTaxID': data_extracted['BuyerTaxID'],
        'Date': data_extracted['Date'],
        'InvoiceNumber': data_extracted['InvoiceNumber'],
        'Seller IBAN': data_extracted['Seller IBAN'],
        'SellerAddress': data_extracted['SellerAddress'],
        'SellerName': data_extracted['SellerName'],
        'SellerTaxID': data_extracted['SellerTaxID'],
        'TotalGrossWorth': data_extracted['TotalGrossWorth'],
        'TotalNetWorth': data_extracted['TotalNetWorth'],
        'TotalVAT': data_extracted['TotalVAT'],
        'TotalVATPercent': data_extracted['TotalVATPercent']
    })

    num_products = len(data_extracted['ProductName'])
    print("NUM PRODUCTS ARE: ", num_products)
    # Iterate through the product-related fields and group them into a dictionary for each product
    list_products = []
    for i in range(num_products):
        product = {
            'ProductName': data_extracted['ProductName'][i],
            'ProductQuantity': data_extracted['ProductQuantity'][i],
            'ProductNetPrice': data_extracted['ProductNetPrice'][i],
            'ProductGrossWorth': data_extracted['ProductGrossWorth'][i],
            'ProductNetWorth': data_extracted['ProductNetWorth'][i],
            'ProductUM': data_extracted['ProductUM'][i],
            'ProductVAT': data_extracted['ProductVAT'][i]
        }
        list_products.append(product)
    json_data['items'] = json.dumps(str(list_products),indent=4, ensure_ascii=False)

    print("\nTHE FINAL DATA EXTRACTED: ", json.dumps(json_data, indent=4, ensure_ascii=False))
    response = jsonify(json_data)
    return response

#cwd = os.getcwd()
#subprocess.run(f'find /home/enflame/OCR_IND/API_issue/test -name "*" 2>&1 | tee {cwd}/test_images.txt', shell=True)

#with open(f'{cwd}/test_images.txt', 'r') as f:
#    imgs = f.readlines()[1:]

#for img in imgs:
#    print("SERVICE STARTED FOR: ", img)
#    result = inference(img.strip())

if __name__ == '__main__':
    app.run(host='localhost', port=5000)

[2024/10/04 10:33:53] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\hp/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\hp/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max

[2024-10-04 10:33:55,381] [    INFO] _internal.py:97 - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:5000
[2024-10-04 10:33:55,382] [    INFO] _internal.py:97 - Press CTRL+C to quit



0: 640x480 1 invoices, 286.4ms
Speed: 18.7ms preprocess, 286.4ms inference, 23.7ms postprocess per image at shape (1, 3, 640, 480)

0: 640x576 1 BuyerAddress, 1 BuyerName, 1 BuyerTaxID, 1 Date, 1 InvoiceNumber, 5 ProductGrossWorths, 5 ProductNames, 5 ProductNetPrices, 5 ProductNetWorths, 5 ProductQuantitys, 5 ProductUMs, 5 ProductVATs, 1 Seller IBAN, 1 SellerAddress, 1 SellerName, 1 SellerTaxID, 1 TotalGrossWorth, 1 TotalNetWorth, 1 TotalVAT, 1 TotalVATPercent, 113.9ms
Speed: 5.0ms preprocess, 113.9ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 576)

OCR Results for InvoiceNumber:
[2024/10/04 10:38:55] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.39743924140930176
[2024/10/04 10:38:56] ppocr DEBUG: cls num  : 1, elapsed : 0.07303309440612793
[2024/10/04 10:38:56] ppocr DEBUG: rec_res num  : 1, elapsed : 0.2710902690887451

OCR Results for Date:
[2024/10/04 10:38:56] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.014991998672485352
[2024/10/04 10:38:56] ppocr DEBUG: cls num  

[2024-10-04 10:39:15,082] [    INFO] _internal.py:97 - 127.0.0.1 - - [04/Oct/2024 10:39:15] "POST /predict HTTP/1.1" 200 -


NUM PRODUCTS ARE:  5

THE FINAL DATA EXTRACTED:  {
    "BuyerAddress": "8113 Hansen Cliff Apt. 826 Port Alice,ID 99663",
    "BuyerName": "Herring-Floyd",
    "BuyerTaxID": "913-80-4636",
    "Date": "07/17/2013",
    "InvoiceNumber": "26343874",
    "items": "\"[{'ProductName': 'Microsoft Xbox One X 1TB 4k Ultra HD Console black controller works qreat1787', 'ProductQuantity': '5,00', 'ProductNetPrice': '249,00', 'ProductGrossWorth': '1369,50', 'ProductNetWorth': '1245,00', 'ProductUM': 'each', 'ProductVAT': '10%'}, {'ProductName': 'Cables 500gb,Sony PlayStation 4 PS4 CUH 1215A Jet Black Console w/Controller', 'ProductQuantity': '5,00', 'ProductNetPrice': '100,00', 'ProductGrossWorth': '550,00', 'ProductNetWorth': '500,00', 'ProductUM': 'each', 'ProductVAT': '10%'}, {'ProductName': 'pS2 Slim Console System SCPH-77000 PINK Playstation 2 Tested', 'ProductQuantity': '4,00', 'ProductNetPrice': '252,00', 'ProductGrossWorth': '1108,80', 'ProductNetWorth': '1008,00', 'ProductUM': 'each', 'Pro

...

NUM PRODUCTS ARE:  5

THE FINAL DATA EXTRACTED:  {
    "BuyerAddress": "8113 Hansen Cliff Apt. 826 Port Alice,ID 99663",
    "BuyerName": "Herring-Floyd",
    "BuyerTaxID": "913-80-4636",
    "Date": "07/17/2013",
    "InvoiceNumber": "26343874",
    "items": "\"[{'ProductName': 'Microsoft Xbox One X 1TB 4k Ultra HD Console black controller works qreat1787', 'ProductQuantity': '5,00', 'ProductNetPrice': '249,00', 'ProductGrossWorth': '1369,50', 'ProductNetWorth': '1245,00', 'ProductUM': 'each', 'ProductVAT': '10%'}, {'ProductName': 'Cables 500gb,Sony PlayStation 4 PS4 CUH 1215A Jet Black Console w/Controller', 'ProductQuantity': '5,00', 'ProductNetPrice': '100,00', 'ProductGrossWorth': '550,00', 'ProductNetWorth': '500,00', 'ProductUM': 'each', 'ProductVAT': '10%'}, {'ProductName': 'pS2 Slim Console System SCPH-77000 PINK Playstation 2 Tested', 'ProductQuantity': '4,00', 'ProductNetPrice': '252,00', 'ProductGrossWorth': '1108,80', 'ProductNetWorth': '1008,00', 'ProductUM': 'each', 'Pro

[2024-10-04 16:04:30,573] [    INFO] _internal.py:97 - 127.0.0.1 - - [04/Oct/2024 16:04:30] "POST /predict HTTP/1.1" 200 -



0: 640x480 1 invoices, 565.4ms
Speed: 109.8ms preprocess, 565.4ms inference, 55.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x576 1 BuyerAddress, 1 BuyerName, 1 BuyerTaxID, 1 Date, 1 InvoiceNumber, 5 ProductGrossWorths, 5 ProductNames, 5 ProductNetPrices, 5 ProductNetWorths, 5 ProductQuantitys, 5 ProductUMs, 5 ProductVATs, 1 Seller IBAN, 1 SellerAddress, 1 SellerName, 1 SellerTaxID, 1 TotalGrossWorth, 1 TotalNetWorth, 1 TotalVAT, 1 TotalVATPercent, 221.1ms
Speed: 9.0ms preprocess, 221.1ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 576)

OCR Results for InvoiceNumber:
[2024/10/04 16:37:25] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.917639970779419
[2024/10/04 16:37:26] ppocr DEBUG: cls num  : 1, elapsed : 0.29604554176330566
[2024/10/04 16:37:26] ppocr DEBUG: rec_res num  : 1, elapsed : 0.8082506656646729

OCR Results for Date:
[2024/10/04 16:37:26] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.017989397048950195
[2024/10/04 16:37:26] ppocr DEBUG: cls num  :

[2024-10-04 16:37:48,215] [    INFO] _internal.py:97 - 127.0.0.1 - - [04/Oct/2024 16:37:48] "POST /predict HTTP/1.1" 200 -


NUM PRODUCTS ARE:  5

THE FINAL DATA EXTRACTED:  {
    "BuyerAddress": "8113 Hansen Cliff Apt. 826 Port Alice,ID 99663",
    "BuyerName": "Herring-Floyd",
    "BuyerTaxID": "913-80-4636",
    "Date": "07/17/2013",
    "InvoiceNumber": "26343874",
    "items": "\"[{'ProductName': 'Microsoft Xbox One X 1TB 4k Ultra HD Console black controller works qreat1787', 'ProductQuantity': '5,00', 'ProductNetPrice': '249,00', 'ProductGrossWorth': '1369,50', 'ProductNetWorth': '1245,00', 'ProductUM': 'each', 'ProductVAT': '10%'}, {'ProductName': 'Cables 500gb,Sony PlayStation 4 PS4 CUH 1215A Jet Black Console w/Controller', 'ProductQuantity': '5,00', 'ProductNetPrice': '100,00', 'ProductGrossWorth': '550,00', 'ProductNetWorth': '500,00', 'ProductUM': 'each', 'ProductVAT': '10%'}, {'ProductName': 'pS2 Slim Console System SCPH-77000 PINK Playstation 2 Tested', 'ProductQuantity': '4,00', 'ProductNetPrice': '252,00', 'ProductGrossWorth': '1108,80', 'ProductNetWorth': '1008,00', 'ProductUM': 'each', 'Pro